## Demo_Match1

Reference:
- [const FLANN_INDEX_HIERARCHICAL](https://docs.opencv.org/3.4/dc/d8c/namespacecvflann.html)

## Run name

In [1]:
import time

project_name = 'Google_LandMark_Rec'
step_name = 'Demo_Match1'
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google_LandMark_Rec_Demo_Match1_20180520_152940


## Important params

In [2]:
# feature_run_name = 'Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180519_141925'
feature_run_name = 'Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411'
print(feature_run_name)

Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os
import sys
import gc
import math
import shutil
import zipfile
import pickle
import h5py
import cv2
from PIL import Image

from tqdm import tqdm
import multiprocessing

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

cpu_amount = multiprocessing.cpu_count()
print('cpu_amount: ', cpu_amount)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


cpu_amount:  36


## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

org_train_folder = os.path.join(input_folder, 'org_train')
org_test_folder = os.path.join(input_folder, 'org_test')
train_folder = os.path.join(input_folder, 'data_train')
val_folder = os.path.join(input_folder, 'data_val')
test_folder = os.path.join(input_folder, 'data_test')
test_sub_folder = os.path.join(test_folder, 'test')

train_csv_file = os.path.join(input_folder, 'train.csv')
test_csv_file = os.path.join(input_folder, 'test.csv')
sample_submission_folder = os.path.join(input_folder, 'sample_submission.csv')

## Preview csv

In [5]:
train_csv = pd.read_csv(train_csv_file)
print('train_csv.shape is {0}.'.format(train_csv.shape))
display(train_csv.head(2))

test_csv = pd.read_csv(test_csv_file)
print('test_csv.shape is {0}.'.format(test_csv.shape))
display(test_csv.head(2))

train_csv.shape is (1225029, 3).


,id,url,landmark_id
0,cacf8152e2d2ae60,http://static.panoramio.com/photos/original/70...,4676
1,0a58358a2afd3e4e,http://lh6.ggpht.com/-igpT6wu0mIA/ROV8HnUuABI/...,6651


test_csv.shape is (117703, 2).


,id,url
0,000088da12d664db,https://lh3.googleusercontent.com/-k45wfamuhT8...
1,0001623c6d808702,https://lh3.googleusercontent.com/-OQ0ywv8KVIA...


In [6]:
train_id = train_csv['id']
train_landmark_id = train_csv['landmark_id']
print('len(train_landmark_id) = \t%s' % len(list(set(train_landmark_id))))

id_2_landmark_id_dict = dict(zip(train_id, train_landmark_id))
print('len(id_2_landmark_id_dict) = \t%d' % len(id_2_landmark_id_dict))

index = 0
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))
index = 1
print('id: %s, \tlandmark_id:%s' % (train_id[index], id_2_landmark_id_dict[train_id[index]]))

len(train_landmark_id) = 	14951
len(id_2_landmark_id_dict) = 	1225029
id: cacf8152e2d2ae60, 	landmark_id:4676
id: 0a58358a2afd3e4e, 	landmark_id:6651


## Load feature

In [7]:
%%time
def dump_pickle_feature_batch(run_name, dataset_name, batch_num, image_features):
    run_name_folder = os.path.join(feature_folder, run_name)
    if not os.path.exists(run_name_folder):
        os.mkdir(run_name_folder)
    image_features_file = os.path.join(run_name_folder, 'feature_%s_%s_b%s.pickle' % (run_name, dataset_name, batch_num))
    print('Dump: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    pickle.dump(image_features, open(image_features_file, "wb"), True)

def load_pickle_feature_batch(run_name, dataset_name, batch_num):
    run_name_folder = os.path.join(feature_folder, run_name)
    image_features_file = os.path.join(run_name_folder, 'feature_%s_%s_b%s.pickle' % (run_name, dataset_name, batch_num))
    image_features = pickle.load(open(image_features_file, "rb"))
    print('Load: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    return image_features

# dump_pickle_feature_batch(run_name, image_features)
image_features = load_pickle_feature_batch(feature_run_name, 'train', 0)

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411_train_b0.pickle  19991
CPU times: user 636 ms, sys: 128 ms, total: 764 ms
Wall time: 1.27 s


In [8]:
print('*'*80)
print('len_image_features=', len(image_features.keys()))
for i, image_id in enumerate(list(image_features.keys())[:3]):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, id_2_landmark_id_dict[image_id]), image_features[image_id].shape)


********************************************************************************
len_image_features= 19991
image_id: 69b846bd58c3f09a, 	landmark_id:6051,	 feature_shape:  (100, 32)
image_id: 19a1de4f08cd0305, 	landmark_id:9179,	 feature_shape:  (100, 32)
image_id: 4ee821754ef5fd83, 	landmark_id:11301,	 feature_shape:  (100, 32)


In [9]:
def load_pickle_feature_batch(image_features_file):
    image_features = pickle.load(open(image_features_file, "rb"))
    print('Load: ', image_features_file, end='  ')
    print(len(image_features.keys()))
    return image_features

In [10]:
def load_pickle_feature_all(feature_run_name, dataset_name):
    file_names = os.listdir(os.path.join(feature_folder, feature_run_name))
    file_names = list(filter(lambda x: dataset_name in x, file_names))[:3]
    all_image_fatures = {}
    for file_name in file_names:
        image_features_file = os.path.join(feature_folder, feature_run_name, file_name)
        image_features = load_pickle_feature_batch(image_features_file)
        all_image_fatures.update(image_features)
    return all_image_fatures

In [11]:
all_image_fatures_train = load_pickle_feature_all(feature_run_name, 'train')

Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411_train_b6.pickle  19996
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411_train_b24.pickle  19993
Load:  /data1/kaggle/landmark-recognition-challenge/feature/Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411/feature_Google_LandMark_Rec_FeatureExtraction_ORB_Parallel_20180520_125411_train_b29.pickle  19990


In [12]:
print('*'*80)
print('len_image_features=', len(all_image_fatures_train.keys()))
for i, image_id in enumerate(list(all_image_fatures_train.keys())[:3]):
    print('image_id: %s, \tlandmark_id:%s,\t feature_shape: ' % (image_id, id_2_landmark_id_dict[image_id]), all_image_fatures_train[image_id].shape)


********************************************************************************
len_image_features= 59979
image_id: 233c663432549b33, 	landmark_id:7048,	 feature_shape:  (100, 32)
image_id: 2e31b117f4748914, 	landmark_id:3249,	 feature_shape:  (100, 32)
image_id: 299f52148392856a, 	landmark_id:4702,	 feature_shape:  (100, 32)


## OrbMatch

In [13]:
%%time

## Basic params

def match(
    feature,
    org_features,
    id_2_landmark_id_dict=id_2_landmark_id_dict,
    is_crossCheck = False, 
    n_matches=50, 
    min_distance=50, 
    min_good_match=30,
    n_class = 14951,
    top=3,
):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=is_crossCheck)
    matched_image_classes = np.zeros(n_class, dtype=np.int16)

    image_ids = list(org_features.keys())
#     rows = len(image_ids)
    rows = 1000
    print(rows)
    for i, image_id in enumerate(image_ids[:rows]):
        test_feature = org_features[image_id]
        if test_feature is None:
            print('None ', image_id, end=' ')
            continue
        try:
            matches = bf.match(feature, org_features[image_id])
        except Exception as ex:
            print(image_id, end=' ')
#             print(ex)
            continue
        matches = list(filter(lambda x: x.distance < min_distance, matches))
        if len(matches) > min_good_match:
            class_idx = id_2_landmark_id_dict[image_id]
            matched_image_classes[class_idx] = matched_image_classes[class_idx] + 1
#             print('image_id=', image_id, end=' ')
#             print('len(matches)=', len(matches), end=' ')
#             print('+%s' % class_idx)
#         else:
#             print('-', end='')

        if (i+1) % 1000 == 0:
            print('%s' % int((i+1)/1000), end=' ')

    print('')
    sorted_matches_idx = np.argsort(matched_image_classes)
    sorted_matches_idx = sorted_matches_idx[::-1]
    best_matches_idx = sorted_matches_idx[0: top]
    best_matches = {}
    for i in best_matches_idx:
        best_matches[i] = matched_image_classes[i]
    return best_matches

indx = 0
finding_key = list(image_features.keys())[indx]
best_matches = match(image_features[finding_key], image_features)
print(best_matches)

# for i in range(1, 3):
#     finding_key = list(image_features.keys())[i]
#     best_matches = match(image_features[finding_key], image_features)
#     print(best_matches)

1000
1 
{6051: 1, 14950: 0, 4993: 0}
CPU times: user 656 ms, sys: 2.56 s, total: 3.21 s
Wall time: 415 ms


In [14]:
%%time

## matches = list(filter(lambda x: x[0].distance < min_distance, matches))

def match_knn(
    feature,
    org_features,
    id_2_landmark_id_dict=id_2_landmark_id_dict,
    is_crossCheck = False, 
    n_matches=100, 
    min_distance=50, 
    min_good_match=30,
    n_class = 14951,
    top=3,
):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    
    
    matched_image_classes = np.zeros(n_class, dtype=np.int16)

    image_ids = list(org_features.keys())
#     rows = len(image_ids)
    rows = 1000
    print(rows)
    for i, image_id in enumerate(image_ids[:rows]):
        test_feature = org_features[image_id]
        if test_feature is None:
            print('None ', image_id, end=' ')
            continue
        try:
            matches = bf.knnMatch(feature, org_features[image_id], k=2)
#             matches = bf.match(feature, org_features[image_id])
        except Exception as ex:
            print(image_id, end=' ')
            print(ex)
            continue
#         print('len(matches)=%s' % len(matches))
#         print('len(matches[0])=%s' % len(matches[0]))
#         print('matches[0][0].distance=%s' % matches[0][0].distance, end=' ')
#         print('matches[0][1].distance=%s' % matches[0][1].distance)
        matches = list(filter(lambda x: x[0].distance < min_distance, matches))
#         matches = list(filter(lambda x: x[0].distance < min_distance and x[0].distance < 0.6*x[1].distance, matches))
#         print('len(matches)=%s' % len(matches))
#         matches = list(filter(lambda x: x.distance < min_distance, matches))
        if len(matches) > min_good_match:
            class_idx = id_2_landmark_id_dict[image_id]
            matched_image_classes[class_idx] = matched_image_classes[class_idx] + 1
#             print('image_id=', image_id, end=' ')
#             print('len(matches)=', len(matches), end=' ')
#             print('+%s' % class_idx)
#         else:
#             print('-', end='')
            
#         print('*'*10)
        if (i+1) % 1000 == 0:
            print('%s' % int((i+1)/1000), end=' ')

    print('')
    sorted_matches_idx = np.argsort(matched_image_classes)
    sorted_matches_idx = sorted_matches_idx[::-1]
    best_matches_idx = sorted_matches_idx[0: top]
    best_matches = {}
    for i in best_matches_idx:
        best_matches[i] = matched_image_classes[i]
    return best_matches

indx = 0
finding_key = list(image_features.keys())[indx]
best_matches = match_knn(image_features[finding_key], image_features)
print(best_matches)

for i in range(1, 3):
    finding_key = list(image_features.keys())[i]
    best_matches = match_knn(image_features[finding_key], image_features)
    print(best_matches)

1000
1 
{6051: 1, 14950: 0, 4993: 0}
1000
1 
{9179: 1, 14950: 0, 4978: 0}
1000
1 
{11301: 1, 4979: 0, 4991: 0}
CPU times: user 2.86 s, sys: 16.3 s, total: 19.2 s
Wall time: 2.68 s


In [15]:
%%time

## matches = list(filter(lambda x: x[0].distance < min_distance and x[0].distance < 0.8*x[1].distance, matches))

def match_knn(
    feature,
    org_features,
    id_2_landmark_id_dict=id_2_landmark_id_dict,
    is_crossCheck = False, 
    n_matches=100, 
    min_distance=50, 
    min_good_match=30,
    n_class = 14951,
    top=3,
):
    bf = cv2.BFMatcher(cv2.NORM_HAMMING)
    
    
    matched_image_classes = np.zeros(n_class, dtype=np.int16)

    image_ids = list(org_features.keys())
#     rows = len(image_ids)
    rows = 1000
    print(rows)
    for i, image_id in enumerate(image_ids[:rows]):
        test_feature = org_features[image_id]
        if test_feature is None:
            print('None ', image_id, end=' ')
            continue
        try:
            matches = bf.knnMatch(feature, org_features[image_id], k=2)
#             matches = bf.match(feature, org_features[image_id])
        except Exception as ex:
            print(image_id, end=' ')
            print(ex)
            continue
#         print('len(matches)=%s' % len(matches))
#         print('len(matches[0])=%s' % len(matches[0]))
#         print('matches[0][0].distance=%s' % matches[0][0].distance, end=' ')
#         print('matches[0][1].distance=%s' % matches[0][1].distance)
        matches = list(filter(lambda x: x[0].distance < min_distance and x[0].distance < 0.8*x[1].distance, matches))
#         matches = list(filter(lambda x: x[0].distance < min_distance and x[0].distance < 0.6*x[1].distance, matches))
#         print('len(matches)=%s' % len(matches))
#         matches = list(filter(lambda x: x.distance < min_distance, matches))
        if len(matches) > min_good_match:
            class_idx = id_2_landmark_id_dict[image_id]
            matched_image_classes[class_idx] = matched_image_classes[class_idx] + 1
#             print('image_id=', image_id, end=' ')
#             print('len(matches)=', len(matches), end=' ')
#             print('+%s' % class_idx)
#         else:
#             print('-', end='')
            
#         print('*'*10)
        if (i+1) % 1000 == 0:
            print('%s' % int((i+1)/1000), end=' ')

    print('')
    sorted_matches_idx = np.argsort(matched_image_classes)
    sorted_matches_idx = sorted_matches_idx[::-1]
    best_matches_idx = sorted_matches_idx[0: top]
    best_matches = {}
    for i in best_matches_idx:
        best_matches[i] = matched_image_classes[i]
    return best_matches

indx = 0
finding_key = list(image_features.keys())[indx]
best_matches = match_knn(image_features[finding_key], image_features)
print(best_matches)

for i in range(1, 3):
    finding_key = list(image_features.keys())[i]
    best_matches = match_knn(image_features[finding_key], image_features)
    print(best_matches)

1000
1 
{6051: 1, 14950: 0, 4993: 0}
1000
1 
{9179: 1, 14950: 0, 4978: 0}
1000
1 
{11301: 1, 4979: 0, 4991: 0}
CPU times: user 2.58 s, sys: 12.6 s, total: 15.2 s
Wall time: 2.7 s


In [26]:
%%time
n_matches = 50

FLANN_INDEX_KDTREE = 0
FLANN_INDEX_HIERARCHICAL = 5
FLANN_INDEX_LSH = 6
index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=10)
search_params = dict(checks=n_matches)
flann = cv2.FlannBasedMatcher(index_params, search_params)

rows = 1000
trained_features = []
for i, image_id in enumerate(list(image_features.keys())[:rows]):
    test_feature = image_features[image_id]
    if test_feature is None:
        print('None ', image_id, end=' ')
        continue
    try:
        trained_features.append(image_features[image_id])
    except Exception as ex:
        print(image_id, end=' ')
        print(ex)
        continue
print('***')
flann.add(trained_features)
print(len(flann.getTrainDescriptors()))
flann.train()

***
1000


error: OpenCV(3.4.1) /io/opencv/modules/flann/src/miniflann.cpp:315: error: (-210) type=0
 in function buildIndex_


In [17]:
# %%time

# ## matches = flann.knnMatch(feature, org_features[image_id], k=2)

# def match_knn(
#     feature,
#     org_features,
#     id_2_landmark_id_dict=id_2_landmark_id_dict,
#     is_crossCheck = False, 
#     n_matches=100, 
#     min_distance=50, 
#     min_good_match=30,
#     n_class = 14951,
#     top=3,
# ):
#     FLANN_INDEX_KDTREE = 0
#     FLANN_INDEX_HIERARCHICAL = 5
#     FLANN_INDEX_LSH = 6
#     index_params = dict(algorithm=FLANN_INDEX_LSH, trees=5)
#     search_params = dict(checks=n_matches)
#     flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    
#     matched_image_classes = np.zeros(n_class, dtype=np.int16)

#     image_ids = list(org_features.keys())
# #     rows = len(image_ids)
#     rows = 1000
#     print(rows)
#     for i, image_id in enumerate(image_ids[:rows]):
#         test_feature = org_features[image_id]
#         if test_feature is None:
#             print('None ', image_id, end=' ')
#             continue
#         try:
#             matches = flann.knnMatch(feature, org_features[image_id], k=2)
# #             matches = bf.match(feature, org_features[image_id])
#         except Exception as ex:
#             print(image_id, end=' ')
#             print(ex)
#             continue
# #         print('len(matches)=%s' % len(matches))
# #         print('len(matches[0])=%s' % len(matches[0]))
# #         print('matches[0][0].distance=%s' % matches[0][0].distance, end=' ')
# #         print('matches[0][1].distance=%s' % matches[0][1].distance)
#         matches = list(filter(lambda x: len(x) > 0 and x[0].distance < min_distance, matches))
# #         matches = list(filter(lambda x: x[0].distance < min_distance and x[0].distance < 0.6*x[1].distance, matches))
# #         print('len(matches)=%s' % len(matches))
# #         matches = list(filter(lambda x: x.distance < min_distance, matches))
#         if len(matches) > min_good_match:
#             class_idx = id_2_landmark_id_dict[image_id]
#             matched_image_classes[class_idx] = matched_image_classes[class_idx] + 1
# #             print('image_id=', image_id, end=' ')
# #             print('len(matches)=', len(matches), end=' ')
# #             print('+%s' % class_idx)
# #         else:
# #             print('-', end='')
            
# #         print('*'*10)
#         if (i+1) % 1000 == 0:
#             print('%s' % int((i+1)/1000), end=' ')

#     print('')
#     sorted_matches_idx = np.argsort(matched_image_classes)
#     sorted_matches_idx = sorted_matches_idx[::-1]
#     best_matches_idx = sorted_matches_idx[0: top]
#     best_matches = {}
#     for i in best_matches_idx:
#         best_matches[i] = matched_image_classes[i]
#     return best_matches

# indx = 0
# finding_key = list(image_features.keys())[indx]
# best_matches = match_knn(image_features[finding_key], image_features)
# print(best_matches)

# for i in range(1, 3):
#     finding_key = list(image_features.keys())[i]
#     best_matches = match_knn(image_features[finding_key], image_features)
#     print(best_matches)

In [18]:
print(len(list(image_features.keys())))

19991


In [19]:
# %%time
# n_matches = 100

# FLANN_INDEX_KDTREE = 0
# FLANN_INDEX_HIERARCHICAL = 5
# FLANN_INDEX_LSH = 6
# index_params = dict(algorithm=FLANN_INDEX_LSH, trees=10)
# search_params = dict(checks=n_matches)
# flann = cv2.FlannBasedMatcher(index_params, search_params)

# rows = 1000
# for i, image_id in enumerate(list(image_features.keys())[:rows]):
#     test_feature = image_features[image_id]
#     if test_feature is None:
#         print('None ', image_id, end=' ')
#         continue
#     try:
#         matches = flann.add(image_features[image_id])
#     except Exception as ex:
#         print(image_id, end=' ')
#         print(ex)
#         continue
# print(len(flann.getTrainDescriptors()))
# flann.train()

In [20]:
# %%time

# ## matches = flann.knnMatch(feature, org_features[image_id], k=2)

# def match_knn(
#     feature,
#     org_features,
#     id_2_landmark_id_dict=id_2_landmark_id_dict,
#     is_crossCheck = False, 
#     n_matches=100, 
#     min_distance=50, 
#     min_good_match=30,
#     n_class = 14951,
#     top=3,
# ):
#     FLANN_INDEX_LSH = 6
#     index_params= dict(
#         algorithm = FLANN_INDEX_LSH,
#         table_number = 6,
#         key_size = 12,
#         multi_probe_level = 1
#     )
#     search_params = dict(n_matches=n_matches)
#     flann = cv2.FlannBasedMatcher(index_params, search_params)
    
    
#     matched_image_classes = np.zeros(n_class, dtype=np.int16)

#     image_ids = list(org_features.keys())
# #     rows = len(image_ids)
#     rows = 1000
#     print(rows)
#     for i, image_id in enumerate(image_ids[:rows]):
#         test_feature = org_features[image_id]
#         if test_feature is None:
#             print('None ', image_id, end=' ')
#             continue
#         try:
#             matches = flann.add(org_features[image_id])
# #             matches = bf.match(feature, org_features[image_id])
#         except Exception as ex:
#             print(image_id, end=' ')
#             print(ex)
#             continue
#     print(len(flann.getTrainDescriptors()))
#     flann.train()
    
#     for i, image_id in enumerate(image_ids[:rows]):
#         test_feature = org_features[image_id]
#         if test_feature is None:
#             print('None ', image_id, end=' ')
#             continue
#         try:
#             matches = flann.add(org_features[image_id])
# #             matches = bf.match(feature, org_features[image_id])
#         except Exception as ex:
#             print(image_id, end=' ')
#             print(ex)
#             continue
#     matches = flann.knnMatch(feature, k=2)
#     print('len(matches)=%s' % len(matches))
#     print('len(matches[0])=%s' % len(matches[0]))

# indx = 0
# finding_key = list(image_features.keys())[indx]
# best_matches = match_knn(image_features[finding_key], image_features)
# # print(best_matches)

# # for i in range(1, 3):
# #     finding_key = list(image_features.keys())[i]
# #     best_matches = match_knn(image_features[finding_key], image_features)
# #     print(best_matches)

In [21]:
gc.collect()

11

## Official data

In [22]:
# split train val dataset
# match --> proba_list
# predcit score

In [23]:
from sklearn.model_selection import train_test_split

image_id_train, image_id_val = train_test_split(list(all_image_fatures_train.keys()), test_size=0.05, random_state=2018)
print('len(image_id_train)=%s' % len(image_id_train))
print('len(image_id_val)=%s' % len(image_id_val))

len(image_id_train)=56980
len(image_id_val)=2999


In [24]:
# from functools import partial

# function_partial = partial(
#     match_knn,
#     org_features=all_image_fatures_train,
#     n_matches=100, 
#     min_distance=30,
#     min_good_match=30,
# )

# image_features_val = [all_image_fatures_train[img_id] for img_id in image_id_val]
# pool = multiprocessing.Pool(processes=cpu_amount)
# for res in tqdm(pool.imap(function_partial, image_features_val), total=len(2)):
#     print(res)
# pool.close()
# pool.terminate()